In [1]:
%run "../head.py"

In [2]:
import pandas as pd
import plotly.express as px
from hic_basic.hicio import load_pickle
from lib.plot import cs_pileup_density, ss_pileup_density

def plot_density_slice(slice_bfs_file, axis, clip, min_samples, **kwargs):
    """
    Reads data from an HDFStore and plots density slices based on specified clips.
    
    Parameters:
        slice_bfs_file (str): Path to the HDFStore file.
        axis (str): The axis along which to slice ('lr', 'dv', 'ht').
        clip: The clip value(s) to plot.
        min_samples (int or list): Minimum number of samples required for plotting.
    """
    other_axis = {
        "lr": ["ht", "dv"],
        "dv": ["ht", "lr"],
        "ht": ["dv", "lr"],
    }[axis]

    with pd.HDFStore(slice_bfs_file, mode="r") as store:
        dat = store.select(
            "main",
            where=f'{axis} == {clip}',
            columns=other_axis + ["sample_name", "density"]
        )
        dat = dat.set_index(
            other_axis + ["sample_name"]
        ).unstack(
            "sample_name"
        ).droplevel(0, axis=1)  # drop "density" itself, make columns flat
        
        # Placeholder for plotting function, needs actual implementation
        # (other_axis) * cells
        fig = ss_pileup_density(
                dat,
                min_samples=min_samples[i] if isinstance(min_samples, list) else min_samples,
                **kwargs
            )
        # fig.update_xaxes(
        #     visible = False,
        # )
        # fig.update_yaxes(
        #     visible = False,
        # )
    return fig

lr_contour = pd.read_pickle(h.fig1 / "Fig1_contour" / f"Sperm.lr_contour.v{h.version}.pkl")
hg_lr_contour = pd.read_pickle(h.fig1 / "Fig1_contour" / f"Sperm_hg.lr_contour.v{h.version}.pkl")

In [3]:
batches = [
    "Sperm","Sperm_hg"
]

bfs_lr_vx_files = {
    batch : (h.ddir / batch).with_suffix(".vx.bfs.h5")
    for batch in batches
}

In [6]:
import pandas as pd

# slice_bfs_files = [bfs_lr_vx_files["Sperm"]]*8 + [bfs_lr_vx_files["Sperm_hg"]]*8
slice_bfs_files = [bfs_lr_vx_files["Sperm"]]*4 + [bfs_lr_vx_files["Sperm_hg"]]*4
axis = "lr"
# clips = [-8, -6 , -4, -2, 0, 2, 4, 6] + [-16, -12, -8, -4, 0, 4, 8, 12]
clips = [-8, -4, 0, 4] + [-12, -6, 0, 6]
min_samples = 0
ext = "pdf"
zmaxes = [30] * 4 + [20] * 4
zmins = [0] * 8
# outfiles = [f"Sperm_lr_{clip}.{ext}" for clip in [-8, -6 , -4, -2, 0, 2, 4, 6]] + \
#               [f"Sperm_hg_lr_{clip}.{ext}" for clip in [-16, -12, -8, -4, 0, 4, 8, 12]]
outfiles = [f"Sperm_lr_{clip}.{ext}" for clip in [-8, -4, 0, 4]] + \
                [f"Sperm_hg_lr_{clip}.{ext}" for clip in [-12, -6, 0, 6]]
for clip, bfs_file, outfile, zmax, zmin in zip(clips, slice_bfs_files, outfiles, zmaxes, zmins):
    fig = plot_density_slice(
        bfs_file, axis, clip, min_samples,
        #showscale = False,
        fill = 0,
        square_size = 50,
        zmin = zmin,
        zmax = zmax,
        showscale = False
        )
    # fig.update_layout(
    #     template = h.svg_heatmap_template,
    #     height = h.svg_heatmap_template.layout.height,
    #     width = h.svg_heatmap_template.layout.width,
    # )
    fig.update_layout(
        template = h.svg_heatmap_template,
        height = h.svg_heatmap_template.layout.height,
        width = h.svg_heatmap_template.layout.width,
        # margin = dict(
        #     l = 0,
        #     r = 0,
        #     b = 0,
        #     t = 0,
        #     pad = 0
        # ),
        title = ""
    )
    fig.update_xaxes(
        tickvals = [-40,0,40],
        ticktext = ["-40","0","40"],
        showticklabels = False,
        mirror = True,
        ticklen = 18
    )
    fig.update_yaxes(
        tickvals = [-40,0,40],
        ticktext = ["-40","0","40"],
        showticklabels = False,
        mirror = True,
        ticklen = 18
    )
    fig.write_image(
        "output/" + outfile
    )